In [1]:
import pandas as pd
import numpy as np
X_train = pd.read_csv('../data/X_train.csv')
Y_train = pd.read_csv('../data/y_train.csv')
X_test = pd.read_csv('../data/X_test.csv')

X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train).iloc[:,1]
X_test = pd.DataFrame(X_test)
X_test_copy = X_test

# 데이터 확인하기
#print(X_test)
#print(X_test.dtypes)
#print(X_test['Customer_care_calls'].unique())

# NA처리하기
# print(X_train.isnull().sum())

# feature selection
X_train = X_train[['Discount_offered','Weight_in_gms']]
X_test = X_test[['Discount_offered','Weight_in_gms']]
# 더비변수 변환
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Feature /target으로 나누기(이미 나눠줘서 나눌 필요 없음)

# 데이터 분할
from sklearn.model_selection import train_test_split
train_feature, test_feature, train_target, test_target = train_test_split(X_train,Y_train,train_size=0.7, random_state=45)

# 데이터 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_feature = scaler.fit_transform(train_feature)
test_feature = scaler.transform(test_feature)
X_test = scaler.transform(X_test)

# 모델링
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
model = RandomForestClassifier(random_state=44)
parameter = {"max_depth":[3,4,5]}
clf = GridSearchCV(model,param_grid=parameter,cv=5)
clf.fit(train_feature,train_target)
clf_best = clf.best_estimator_

# 정확도 예측
from sklearn.metrics import roc_auc_score
pred_1 = clf_best.predict(test_feature)
print(roc_auc_score(test_target,pred_1)) # feature_selection을 안했을 경우 : 0.73 / ID를 제거했을 때 : 0.69 / 상관계수 높은것만 : 0.67

# 결과값 to_csv로 내보내기
pred = clf_best.predict_proba(X_test)[:,1]
print(pred)
a = pd.DataFrame({'cust_id' : X_test_copy['ID'], 'Predict': pred})# .to_csv('0030000.csv',index=False)

KeyError: "None of [Index(['Discount_offered', 'Weight_in_gms'], dtype='object')] are in the [columns]"

In [115]:
df = pd.concat([X_train,Y_train],axis=1)
df.corr().iloc[:,-1]

Discount_offered       0.395611
Weight_in_gms         -0.271201
Reached.on.Time_Y.N    1.000000
Name: Reached.on.Time_Y.N, dtype: float64

In [114]:
a = pd.DataFrame({'cust_id' : X_test_copy['ID'], 'Predict': pred})# .to_csv('0030000.csv',index=False)
b = pd.read_csv('./test_label/y_test.csv')
b = pd.DataFrame(b)

pred_ex = clf_best.predict(X_test)
b = b.iloc[:,1]

print(roc_auc_score(b,pred_ex)) # featuring 안했을 때 : 0.72 / ID를 제거했을 떄 : 0.69 / 상관계수 높은것만 : 0.68

0.6861480535919247
